In [1]:
import ee
import geemap
import os


In [2]:
# geemap.update_package()

In [3]:
Map = geemap.Map()


In [4]:
mosaico = ee.Image("users/franciscoalbornoz/Mosaico_2018")
clasificacion_2018 = ee.Image("users/franciscoalbornoz/clasificacion_2018")
landcover_2018 = ee.Image("users/franciscoalbornoz/landcover_sur")
landsat_vis = {
    'min': 0,
    'max': 1,
    'bands': ['b5', 'b4', 'b3']
}
Map.addLayer(mosaico, landsat_vis, "Landsat-8-2018")
Map.addLayer(clasificacion_2018.randomVisualizer(), {}, "clasificación 2018")
Map.addLayer(landcover_2018.randomVisualizer(), {} , "landcover provincia")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [36]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [37]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [7]:
landcover_2018.bandNames().getInfo()

['b1']

In [8]:
clasificacion_2018.bandTypes().getInfo()


{'b1': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}}

In [9]:
# land2018 = landcover_2018.set('classification_class_values', class_values)
# land2018f = land2018.set('classification_class_palette', class_palette)

In [10]:
# class2018 = clasificacion_2018.set('classification_class_values', class_values)
# class2018f = class2018.set('classification_class_palette', class_palette)

In [11]:
# Map.addLayer(land2018f, {}, 'landcover_prov_coyhaique' )
# Map.addLayer(class2018f,{},'clasificacion 2018')

In [12]:
mosaico.bandNames().getInfo()

['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [13]:
work_dir = os.path.expanduser('../Data/shapes/')
achaparrado_shp = os.path.join(work_dir, 'achaparrado_2018_2210_grande.shp')
agricolas_shp = os.path.join(work_dir, 'agricolas_2018_2610_grande.shp')
agua_shp = os.path.join(work_dir, 'agua_2018_0310_grande.shp')
arenalroca_shp = os.path.join(work_dir, 'arenal_roca_2018_0310_grande.shp')
bosquedegradado_shp = os.path.join(work_dir, 'bosques_degradados_2018_22112018.shp')
cuncuna_shp = os.path.join(work_dir, 'cuncunas_2018_2210_grande.shp')
estepa_shp = os.path.join(work_dir, 'estepa_2018_0310_grande.shp')
humedal_shp = os.path.join(work_dir, 'humedal_agua_2018_0310_grande.shp')
mallin_shp = os.path.join(work_dir, 'mallin_2018_0510_grande.shp')
matorralabierto_shp = os.path.join(work_dir, 'matorrales_abiertos_2018_22102018.shp')
matorraldenso_shp = os.path.join(work_dir, 'matorrales_densos_2018_22102018.shp')
plantacion1_shp = os.path.join(work_dir, 'plantaciones_2018_0310_grande.shp')
plantacion2_shp = os.path.join(work_dir, 'plantaciones_2018_23112018_grande.shp')
plantacion3_shp = os.path.join(work_dir, 'plantaciones_2018_26112018_grande.shp')
pradera_shp = os.path.join(work_dir, 'praderas_2018_0310_grande.shp')
primario_shp = os.path.join(work_dir, 'primario_2018_0310_grande.shp')
secundario_shp = os.path.join(work_dir, 'secundario_2018_0310_grande.shp')
sombra_shp = os.path.join(work_dir, 'sombras_topograficas_2018_0310_grande.shp')
suelo_shp = os.path.join(work_dir, 'suelo_desnudo_agricola_2018_0310_grande.shp')
merged_shp = os.path.join(work_dir, 'roi_merged.shp')

In [14]:
merged =geemap.shp_to_ee(merged_shp)
Map.addLayer(merged, {}, 'Áreas de entrenamiento')

In [15]:
# poligono = ee.FeatureCollection([
#     ee.Feature(achaparrado),
#     ee.Feature(agricolas),
#     ee.Feature(agua),
#     ee.Feature(arenalroca),
#     ee.Feature(bosquedegradado),
#     ee.Feature(cuncuna),
#     ee.Feature(estepa),
#     ee.Feature(humedal),
#     ee.Feature(mallin),
#     ee.Feature(matorralabierto),
#     ee.Feature(matdenso),
#     ee.Feature(plantacion1),
#     ee.Feature(plantacion2),
#     ee.Feature(plantacion3),
#     ee.Feature(pradera),
#     ee.Feature(primario),
#     ee.Feature(secundario),
#     ee.Feature(sombra),
#     ee.Feature(suelo),    
# ])


In [16]:
merged_shp

'../Data/shapes/roi_merged.shp'

In [17]:
dataset = mosaico.sampleRegions(**{
    'collection': merged,
    'properties': ['ID'],
    'scale' : 30 ,
})

In [18]:
dataset.size().getInfo()

14254

In [19]:
print(dataset.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 1, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173}}


In [20]:
# Guardamos las bandas
bands = ['b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

# Esta propiedad guarda las clases de landcover
label = 'ID'

# Overlay the points on the imagery to get training.
# sample = mosaico.select(bands).sampleRegions(**{
#   'collection': dataset,
#   'properties': ['RF'],
#   'scale': 30
# })

# Adds a column of deterministic pseudorandom numbers. 
sample = dataset.randomColumn()

split = 0.7 

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

In [21]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_1', 'properties': {'ID': 1, 'b1': 0.02466532774269581, 'b2': 0.03902462124824524, 'b3': 0.03005797602236271, 'b4': 0.31321027874946594, 'b5': 0.13090473413467407, 'b6': 0.061290204524993896, 'b7': 0.8248717784881592, 'b8': 0.7784170508384705, 'b9': 0.5036694407463074, 'random': 0.16340030157546792}}


In [22]:
print(validation.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'ID': 1, 'b1': 0.026168961077928543, 'b2': 0.04253531992435455, 'b3': 0.03470490872859955, 'b4': 0.33766427636146545, 'b5': 0.1495322287082672, 'b6': 0.06873124837875366, 'b7': 0.8135994672775269, 'b8': 0.7762474417686462, 'b9': 0.5209251642227173, 'random': 0.9956806624347441}}


In [23]:
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

In [24]:
# Clasificando con las mismas bandas de entrenamiento
result = mosaico.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classificado')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [25]:
result.getInfo()

{'type': 'Image',
 'bands': [{'id': 'classification',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': -2147483648,
    'max': 2147483647},
   'dimensions': [6083, 9750],
   'crs': 'EPSG:32719',
   'crs_transform': [30, 0, 179085, 0, -30, 5153095]}],
 'properties': {'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-70.71978119886894, -43.7621302946721],
    [-70.71984230426219, -43.762120959847216],
    [-72.98275024455971, -43.70561719895318],
    [-72.98292936081644, -43.70572079387638],
    [-72.98313712829581, -43.705812701067146],
    [-73.02744506941205, -44.362573403829224],
    [-73.07328396187123, -45.01916085040947],
    [-73.12072282029395, -45.67563442548405],
    [-73.16983051350653, -46.331992631080205],
    [-73.16968129990005, -46.33212249547371],
    [-73.1695568677538, -46.33226430394552],
    [-73.16949325899127, -46.33227507995767],
    [-70.80072781794739, -46.3941742696168],
    [-70.80054376625334, -46.39406743407541],
    [-

In [26]:
class_values = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
    ]

In [27]:
class_palette = ['22c129',
 'f0ff00',
 '0c40f1',
 '685205',
 '0d4404',
 'e82d13',
 'f99504',
 '17d4c6',
 '1c6330',
 '24ff00',
 '2de50f',
 'ccba7d',
 'dec61a',
 '28d909',
 '03c2ff',
 '4e8c44',
 'fdfdfd',
 'c0c233'
]

In [38]:
land = result.set('classification_class_values', class_values)
landcoverf = land.set('classification_class_palette', class_palette)

In [40]:
Map.addLayer(landcoverf, {}, 'Land cover Random Forest')
Map

Map(bottom=2698324.0, center=[-45.75458888849699, -72.05035686492921], controls=(WidgetControl(options=['posit…

In [30]:
train_accuracy = classifier.confusionMatrix()

In [31]:
train_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 419, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 553, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 2137, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1121, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 69, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 651, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 2, 0, 0, 420, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 4, 0, 0, 0, 0, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 61, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 153, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 1, 109, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2137, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 188, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 544, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 149, 0, 0, 0],
 [0, 7, 0,

In [32]:
train_accuracy.accuracy().getInfo()

0.9968756299133239

In [33]:
train_accuracy.kappa().getInfo()

0.9964227017790738

In [34]:
train_accuracy.producersAccuracy().getInfo()

[[0],
 [0.9976190476190476],
 [0.9981949458483754],
 [1],
 [0.9991087344028521],
 [0.9583333333333334],
 [1],
 [0.995260663507109],
 [0.8947368421052632],
 [1],
 [1],
 [0.956140350877193],
 [0.9990649836372136],
 [0.9947089947089947],
 [0.9945155393053017],
 [1],
 [0.9854166666666667],
 [1],
 [0.9982394366197183]]

In [35]:
train_accuracy.consumersAccuracy().getInfo()

[[0,
  0.9812646370023419,
  0.9981949458483754,
  0.9981317141522653,
  0.9982190560997328,
  0.9857142857142858,
  0.9923780487804879,
  1,
  1,
  0.9838709677419355,
  0.9935064935064936,
  0.9646017699115044,
  0.9990649836372136,
  1,
  1,
  1,
  0.9978902953586498,
  1,
  0.9982394366197183]]

In [41]:
validacion = validation.classify(classifier)

In [42]:
testAccuracy = validacion.errorMatrix('ID', 'classification')

In [43]:
testAccuracy.accuracy().getInfo()

0.9727608494921515